In [1]:
import cv2
import tensorflow as tf
import numpy as np
import os



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/

In [2]:
big_image_size = 1000
hr_size = 100
lr_size = 50


In [3]:
def big_image_to_parts(img, hr_size, big_image_size):
    r_image = cv2.resize(img, (big_image_size, big_image_size))
    height, width, channel = r_image.shape
    CROP_H_SIZE = big_image_size // hr_size
    CROP_W_SIZE = big_image_size // hr_size
    images = []
    for ih in range(CROP_H_SIZE):
        for iw in range(CROP_W_SIZE):
            x = width // CROP_W_SIZE * iw
            y = height // CROP_H_SIZE * ih
            h = (height // CROP_H_SIZE)
            w = (width // CROP_W_SIZE)
            images.append(r_image[y:y + h, x:x + w])
    return images


In [4]:
X = []
y = []
file_index = 1
for file in os.listdir("pics/"):
    file_image = cv2.imread("pics/" + file, cv2.IMREAD_COLOR)
    pic_index = 1
    small_images = big_image_to_parts(file_image, hr_size, big_image_size)
    for img in small_images:
        pic_name = "input_images/y/" + str(file_index) + "_" + str(pic_index) + ".jpg"
        y.append(img)
        cv2.imwrite(pic_name, img)
        img_scale_down = cv2.resize(img, (lr_size, lr_size))
        image2 = cv2.resize(img_scale_down, (hr_size, hr_size), interpolation=cv2.INTER_LINEAR)
        X.append(image2)
        pic_name = "input_images/X/" + str(file_index) + "_" + str(pic_index) + ".jpg"
        cv2.imwrite(pic_name, image2)
        pic_index += 1
    file_index += 1


In [5]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models
import sklearn 

X = np.array(X)
y = np.array(y)

input = layers.Input(shape=(hr_size, hr_size, 3))
x = layers.Conv2D(32, (9, 9), padding="same", activation="relu", name="level1")(input)
x = layers.Conv2D(32, (5, 5), padding='same', activation="relu", name="level2")(x)
out = layers.Conv2D(3, (3, 3), padding='same', activation="relu", name="out")(x)

model = keras.Model(input, out)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
X.shape


(10000, 100, 100, 3)

In [7]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 100, 3)]     0         
_________________________________________________________________
level1 (Conv2D)              (None, 100, 100, 32)      7808      
_________________________________________________________________
level2 (Conv2D)              (None, 100, 100, 32)      25632     
_________________________________________________________________
out (Conv2D)                 (None, 100, 100, 3)       867       
Total params: 34,307
Trainable params: 34,307
Non-trainable params: 0
_________________________________________________________________
